# Idea: ABcache
## getting a precomputed ABcache file containing the parameters for beta-binomial distribution from the PoN-list

# Running EBfilter createCache on testdata

## setting the config

In [ ]:
cd ..

In [ ]:
from codes import run

# snakemake config
config = {'EB':{'run': True}}
params = {}
params['map_quality'] = 20
params['base_quality'] = 15
params['filter_flags'] = 'UNMAP,SECONDARY,QCFAIL,DUP'
params['fitting_penalty'] = .5
params['caching'] = True
# to simulate snakemake behavior
config['EB']['threads'] = 1
config['EB']['params'] = params
config['EB']
config['annovar'] = {'sep': '\t'}
config['EB']['log'] = 'output/logfile'


params = config['EB']['params']
threads = config['EB']['threads']
log = config['EB']['log']
sep = config['annovar']['sep']
_q = str(params['map_quality'])  # mapping quality=20
_Q = params['base_quality']      # base quality=15
fit_pen = params['fitting_penalty']
filter_quals = ''
for qual in range( 33, 33 + _Q ): 
    filter_quals += chr( qual )  # qual asciis for filtering out
_ff = params['filter_flags']     # 'UNMAP,SECONDARY,QCFAIL,DUP'
config = {'q':_q, 'Q':_Q, 'filter_quals': filter_quals, 'log':log, 'fitting_penalty': fit_pen, 'ff':_ff, 'threads':threads, 'sep': sep}


### running makeEBcache on testdata

In [ ]:
args = {}
args['pon_list'] = 'testdata/PoN_list.txt'
args['cache_folder'] = 'output/testdata_cache' # provide a folder for storing the data (snakemake config)
args['force_caching'] = True                  # force cache generation although no bed_file is provided
args['bed_file'] = 'testdata/input.bed'
args['generate_cache'] = True

In [ ]:
config['threads'] = 3
config['debug_mode'] = False
run.main(args, config)
!ls output

### running EBscore in cache_mode on testdata

In [ ]:
args = {}
args['mut_file'] = 'testdata/input.anno'
args['tumor_bam'] = 'testdata/tumor.bam'
args['pon_list'] = 'testdata/PoN_list.txt'
args['output_path'] = 'output/testdata_eb.csv'
args['cache_folder'] = 'output/testdata_cache'
args['generate_cache'] = False

In [ ]:
config['threads'] = 3
config['debug_mode'] = False
run.main(args, config)
!ls output

# Running EBfilter createCache on my data

### Setting my testdata as arguments

In [ ]:
import os
HOME = os.environ['HOME'] # set HOME to run on different Macs
args = {}
args['pon_list'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list'
args['cache_folder'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.ABcache'
args['force_caching'] = True
args['bed_file'] = None
args['generate_cache'] = True

In [ ]:
config['threads'] = 3
config['debug_mode'] = True
run.main(args, config)
!ls output

# Running EBfilter in Cache mode on my data

In [ ]:
HOME = os.environ['HOME']
args['pon_list'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list'
args['output_path'] = 'output/test_rel_eb.csv'
args['region'] = ''
log_file = f"{os.path.splitext(args['output_path'])[0]}.log"
state['log'] = log_file                   

### apply a column apply on several columns

In [ ]:
import re
sign_re = re.compile(r'\^.|\$')
def clean_reads(run_column):

    return run_column.str.replace(sign_re, '')


pileup_small[['read0', 'read1']] = pileup_small[['read0', 'read1']].apply(lambda column: column.str.replace(sign_re, ''))
